In [1]:
import time
import numpy as np
import tensorflow as tf

In [16]:
import reader

In [21]:
!wget http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz
!tar xzf simple-examples.tgz -C data/

SYSTEM_WGETRC = c:/progra~1/wget/etc/wgetrc
syswgetrc = c:/progra~1/wget/etc/wgetrc
--2020-05-23 11:27:32--  http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz
Resolving www.fit.vutbr.cz... 147.229.9.23
Connecting to www.fit.vutbr.cz|147.229.9.23|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34869662 (33M) [application/x-gtar]
Saving to: `simple-examples.tgz.1'

     0K .......... .......... .......... .......... ..........  0%  141K 4m1s
    50K .......... .......... .......... .......... ..........  0%  282K 3m0s
   100K .......... .......... .......... .......... ..........  0%  306K 2m37s
   150K .......... .......... .......... .......... ..........  0%  291K 2m27s
   200K .......... .......... .......... .......... ..........  0%  321K 2m18s
   250K .......... .......... .......... .......... ..........  0% 1.59M 1m58s
   300K .......... .......... .......... .......... ..........  1%  346K 1m55s
   350K .......... .......... .......... .......

In [18]:
init_scale = 0.1
learning_rate = 1.0
max_grad_norm = 5
num_layers = 2
num_steps = 20
hidden_size_l1 = 256
hidden_size_l2 = 128
max_epoch_decay_lr = 4
max_epoch = 15
keep_prob = 1
decay = 0.5
batch_size = 60
vocab_size = 10000
embedding_vector_size = 200
is_training = 1
data_dir = 'data/simple-examples/data/'

In [22]:
session = tf.InteractiveSession()
raw_data = reader.ptb_raw_data(data_dir)
train_data, valid_data, test_data, vocab, word_to_id = raw_data

C:\Source-Files\anaconda3\envs\old_tensorflow_1.15\lib\site-packages\tensorflow_core\python\client\session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [23]:
len(train_data)

929589

In [25]:
def id_to_word(id_list):
    line = []
    for w in id_list:
        for word, wid in word_to_id.items():
            if wid == w:
                line.append(word)
    return line
id_to_word(train_data[0:100])

['aer',
 'banknote',
 'berlitz',
 'calloway',
 'centrust',
 'cluett',
 'fromstein',
 'gitano',
 'guterman',
 'hydro-quebec',
 'ipo',
 'kia',
 'memotec',
 'mlx',
 'nahb',
 'punts',
 'rake',
 'regatta',
 'rubens',
 'sim',
 'snack-food',
 'ssangyong',
 'swapo',
 'wachter',
 '<eos>',
 'pierre',
 '<unk>',
 'N',
 'years',
 'old',
 'will',
 'join',
 'the',
 'board',
 'as',
 'a',
 'nonexecutive',
 'director',
 'nov.',
 'N',
 '<eos>',
 'mr.',
 '<unk>',
 'is',
 'chairman',
 'of',
 '<unk>',
 'n.v.',
 'the',
 'dutch',
 'publishing',
 'group',
 '<eos>',
 'rudolph',
 '<unk>',
 'N',
 'years',
 'old',
 'and',
 'former',
 'chairman',
 'of',
 'consolidated',
 'gold',
 'fields',
 'plc',
 'was',
 'named',
 'a',
 'nonexecutive',
 'director',
 'of',
 'this',
 'british',
 'industrial',
 'conglomerate',
 '<eos>',
 'a',
 'form',
 'of',
 'asbestos',
 'once',
 'used',
 'to',
 'make',
 'kent',
 'cigarette',
 'filters',
 'has',
 'caused',
 'a',
 'high',
 'percentage',
 'of',
 'cancer',
 'deaths',
 'among',
 'a',
 

In [26]:
itera = reader.ptb_iterator(train_data, batch_size, num_steps)
first_touple = itera.__next__()
x = first_touple[0]
y = first_touple[1]

In [27]:
x.shape

(60, 20)

In [28]:
x[0:3]

array([[9970, 9971, 9972, 9974, 9975, 9976, 9980, 9981, 9982, 9983, 9984,
        9986, 9987, 9988, 9989, 9991, 9992, 9993, 9994, 9995],
       [ 901,   33, 3361,    8, 1279,  437,  597,    6,  261, 4276, 1089,
           8, 2836,    2,  269,    4, 5526,  241,   13, 2420],
       [2654,    6,  334, 2886,    4,    1,  233,  711,  834,   11,  130,
         123,    7,  514,    2,   63,   10,  514,    8,  605]])

In [30]:
_input_data = tf.placeholder(tf.int32, [batch_size, num_steps])
_targets = tf.placeholder(tf.int32, [batch_size, num_steps])

In [33]:
feed_dict = {_input_data: x, _targets: y}

In [34]:
session.run(_input_data, feed_dict)

array([[9970, 9971, 9972, ..., 9993, 9994, 9995],
       [ 901,   33, 3361, ...,  241,   13, 2420],
       [2654,    6,  334, ...,  514,    8,  605],
       ...,
       [7831,   36, 1678, ...,    4, 4558,  157],
       [  59, 2070, 2433, ...,  400,    1, 1173],
       [2097,    3,    2, ..., 2043,   23,    1]])

In [35]:
lstm_cell_l1 = tf.contrib.rnn.BasicLSTMCell(hidden_size_l1, forget_bias = 0.0)
lstm_cell_l2 = tf.contrib.rnn.BasicLSTMCell(hidden_size_l2, forget_bias = 0.0)
stacked_lstm = tf.contrib.rnn.MultiRNNCell([lstm_cell_l1, lstm_cell_l2])

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.


In [36]:
_initial_state = stacked_lstm.zero_state(batch_size, tf.float32)
_initial_state

(LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState/zeros:0' shape=(60, 256) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState/zeros_1:0' shape=(60, 256) dtype=float32>),
 LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState_1/zeros:0' shape=(60, 128) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState_1/zeros_1:0' shape=(60, 128) dtype=float32>))

In [37]:
session.run(_initial_state, feed_dict)

(LSTMStateTuple(c=array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), h=array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)),
 LSTMStateTuple(c=array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), h=array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
  

In [38]:
embedding_vocab = tf.get_variable('embedding_vocab', [vocab_size, embedding_vector_size])

In [39]:
session.run(tf.global_variables_initializer())
session.run(embedding_vocab)

array([[ 0.00774599, -0.00554714,  0.01253956, ...,  0.01408619,
         0.0141963 , -0.01522163],
       [ 0.01077044,  0.00271998,  0.01717146, ..., -0.01202794,
         0.01830959, -0.01185397],
       [ 0.01874628,  0.01186094,  0.00947465, ..., -0.01714887,
         0.02346812,  0.02298217],
       ...,
       [ 0.00696038,  0.02371449, -0.0122404 , ...,  0.01838088,
         0.01253957,  0.02226672],
       [ 0.00339366,  0.0184164 , -0.01297108, ...,  0.02242959,
        -0.00364506, -0.01650954],
       [-0.0030819 ,  0.01843034,  0.00818174, ..., -0.01524403,
        -0.00527325, -0.00615346]], dtype=float32)

In [40]:
inputs = tf.nn.embedding_lookup(embedding_vocab, _input_data)
inputs

<tf.Tensor 'embedding_lookup/Identity:0' shape=(60, 20, 200) dtype=float32>

In [41]:
session.run(inputs[0], feed_dict)

array([[-0.0113657 ,  0.01950674, -0.01097114, ..., -0.01661037,
        -0.01597887, -0.0112855 ],
       [ 0.0029857 , -0.01817202, -0.02336918, ..., -0.00681213,
         0.01696006,  0.00200827],
       [ 0.01614475, -0.00163903, -0.02294274, ...,  0.00502876,
         0.00407615,  0.00937419],
       ...,
       [ 0.00985015, -0.02407787,  0.00887106, ...,  0.02028637,
        -0.01327894, -0.01149644],
       [ 0.01556183, -0.0071445 ,  0.01149855, ...,  0.00436933,
        -0.01731777, -0.02096438],
       [-0.02156661,  0.00825505, -0.0056548 , ..., -0.01175688,
         0.00551854,  0.02293842]], dtype=float32)

In [42]:
outputs, new_state = tf.nn.dynamic_rnn(stacked_lstm, inputs, initial_state = _initial_state)

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [43]:
outputs

<tf.Tensor 'rnn/transpose_1:0' shape=(60, 20, 128) dtype=float32>

In [44]:
session.run(tf.global_variables_initializer())
session.run(outputs[0], feed_dict)

array([[ 3.84070998e-04, -3.58524383e-04,  1.21615594e-04, ...,
         3.06471338e-05,  6.58223464e-04, -1.25377584e-04],
       [ 2.99432490e-04, -2.07678386e-04,  2.81619868e-04, ...,
        -1.08400745e-04,  4.71692852e-04, -9.61323440e-06],
       [ 3.30161973e-04, -1.34147747e-04, -5.10570477e-04, ...,
         1.90621635e-04,  1.41600409e-04,  3.36510915e-04],
       ...,
       [-2.20630376e-04, -3.57483252e-04,  5.73305588e-05, ...,
        -1.51926593e-03, -1.31620429e-04, -1.33322051e-03],
       [ 3.25640664e-04, -4.90527200e-05, -1.85516008e-04, ...,
        -1.95999490e-03,  6.84425118e-04, -8.28667893e-04],
       [ 6.94147311e-04,  9.54036950e-04, -6.71346788e-05, ...,
        -1.09706342e-03,  1.36941031e-03, -8.80682259e-04]], dtype=float32)

In [45]:
output = tf.reshape(outputs, [-1, hidden_size_l2])
output

<tf.Tensor 'Reshape:0' shape=(1200, 128) dtype=float32>

In [46]:
softmax_w = tf.get_variable('softmax_w', [hidden_size_l2, vocab_size])
softmax_b = tf.get_variable('softmax_b', [vocab_size])
logits = tf.matmul(output, softmax_w) + softmax_b
prob = tf.nn.softmax(logits)

In [47]:
session.run(tf.global_variables_initializer())
output_words_prob = session.run(prob, feed_dict)
output_words_prob.shape, output_words_prob[:20]

((1200, 10000),
 array([[1.01449514e-04, 1.01507663e-04, 1.00352889e-04, ...,
         1.01337908e-04, 9.94753063e-05, 1.01354199e-04],
        [1.01459053e-04, 1.01511694e-04, 1.00345314e-04, ...,
         1.01335703e-04, 9.94794755e-05, 1.01351085e-04],
        [1.01455837e-04, 1.01510137e-04, 1.00344587e-04, ...,
         1.01336067e-04, 9.94736547e-05, 1.01353937e-04],
        ...,
        [1.01456179e-04, 1.01508420e-04, 1.00360456e-04, ...,
         1.01341648e-04, 9.94752118e-05, 1.01360944e-04],
        [1.01451908e-04, 1.01510566e-04, 1.00358047e-04, ...,
         1.01347192e-04, 9.94788788e-05, 1.01363010e-04],
        [1.01456244e-04, 1.01506630e-04, 1.00359473e-04, ...,
         1.01356592e-04, 9.94763468e-05, 1.01366277e-04]], dtype=float32))

In [49]:
np.argmax(output_words_prob[:20], axis = 1)

array([2811, 1759, 1759, 7746, 7746, 7746, 9863, 9863, 9863, 5426, 5426,
       5426, 9667, 9667, 9667, 9667, 1898, 8124, 2232, 9872], dtype=int64)

In [50]:
y[0]

array([9971, 9972, 9974, 9975, 9976, 9980, 9981, 9982, 9983, 9984, 9986,
       9987, 9988, 9989, 9991, 9992, 9993, 9994, 9995, 9996])

In [51]:
targ = session.run(_targets, feed_dict)
targ[0]

array([9971, 9972, 9974, 9975, 9976, 9980, 9981, 9982, 9983, 9984, 9986,
       9987, 9988, 9989, 9991, 9992, 9993, 9994, 9995, 9996])

In [52]:
loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example([logits], [tf.reshape(_targets, [-1])], [tf.ones([batch_size * num_steps])])

In [53]:
session.run(loss, feed_dict)[:10]

array([9.219143, 9.201125, 9.209398, 9.215267, 9.223923, 9.207102,
       9.2074  , 9.20625 , 9.208804, 9.21862 ], dtype=float32)

In [54]:
cost = tf.reduce_sum(loss) / batch_size
session.run(tf.global_variables_initializer())
session.run(cost, feed_dict)

184.22784

In [55]:
lr = tf.Variable(0.0, trainable = False)
optimizer = tf.train.GradientDescentOptimizer(lr)

In [56]:
tvars = tf.trainable_variables()
tvars

[<tf.Variable 'embedding_vocab:0' shape=(10000, 200) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel:0' shape=(456, 1024) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_0/basic_lstm_cell/bias:0' shape=(1024,) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_1/basic_lstm_cell/kernel:0' shape=(384, 512) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_1/basic_lstm_cell/bias:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'softmax_w:0' shape=(128, 10000) dtype=float32_ref>,
 <tf.Variable 'softmax_b:0' shape=(10000,) dtype=float32_ref>]

In [57]:
[v.name for v in tvars]

['embedding_vocab:0',
 'rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel:0',
 'rnn/multi_rnn_cell/cell_0/basic_lstm_cell/bias:0',
 'rnn/multi_rnn_cell/cell_1/basic_lstm_cell/kernel:0',
 'rnn/multi_rnn_cell/cell_1/basic_lstm_cell/bias:0',
 'softmax_w:0',
 'softmax_b:0']

In [58]:
var_x = tf.placeholder(tf.float32)
var_y = tf.placeholder(tf.float32)
func_test = 2.0 * var_x * var_x + 3.0 * var_x * var_y
session.run(tf.global_variables_initializer())
session.run(func_test, {var_x: 1.0, var_y: 2.0})

8.0

In [59]:
var_grad = tf.gradients(func_test, [var_x])
session.run(var_grad, {var_x: 1.0, var_y: 2.0})

[10.0]

In [60]:
var_grad = tf.gradients(func_test, [var_y])
session.run(var_grad, {var_x: 1.0, var_y: 2.0})

[3.0]

In [61]:
tf.gradients(cost, tvars)

 <tf.Tensor 'gradients_2/rnn/while/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/MatMul/Enter_grad/b_acc_3:0' shape=(456, 1024) dtype=float32>,
 <tf.Tensor 'gradients_2/rnn/while/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/BiasAdd/Enter_grad/b_acc_3:0' shape=(1024,) dtype=float32>,
 <tf.Tensor 'gradients_2/rnn/while/rnn/multi_rnn_cell/cell_1/basic_lstm_cell/MatMul/Enter_grad/b_acc_3:0' shape=(384, 512) dtype=float32>,
 <tf.Tensor 'gradients_2/rnn/while/rnn/multi_rnn_cell/cell_1/basic_lstm_cell/BiasAdd/Enter_grad/b_acc_3:0' shape=(512,) dtype=float32>,
 <tf.Tensor 'gradients_2/MatMul_grad/MatMul_1:0' shape=(128, 10000) dtype=float32>,
 <tf.Tensor 'gradients_2/add_grad/Reshape:0' shape=(10000,) dtype=float32>]

In [63]:
grad_t_list = tf.gradients(cost, tvars)

In [65]:
grads, _ = tf.clip_by_global_norm(grad_t_list, max_grad_norm)
grads

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


 <tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_1:0' shape=(456, 1024) dtype=float32>,
 <tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_2:0' shape=(1024,) dtype=float32>,
 <tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_3:0' shape=(384, 512) dtype=float32>,
 <tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_4:0' shape=(512,) dtype=float32>,
 <tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_5:0' shape=(128, 10000) dtype=float32>,
 <tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_6:0' shape=(10000,) dtype=float32>]

In [66]:
session.run(grads, feed_dict)

[IndexedSlicesValue(values=array([[ 8.1478547e-06,  2.0814894e-06, -8.1156595e-06, ...,
         -5.6041822e-06,  1.1665610e-05, -7.1158474e-06],
        [ 8.4184248e-06,  5.0070580e-06,  4.3879040e-06, ...,
         -5.1715842e-06,  7.8525291e-06, -1.3794806e-05],
        [ 5.4588022e-06, -1.3837293e-06,  1.0301756e-05, ...,
          8.9897333e-07,  4.3825403e-06, -1.7853530e-05],
        ...,
        [ 4.0232890e-06, -4.3145127e-07,  2.5169993e-06, ...,
         -1.2319755e-06,  3.0322983e-06, -3.5352734e-06],
        [ 6.8481045e-06,  3.4138786e-06, -7.4687489e-07, ...,
         -4.1681571e-07,  6.7079254e-06,  5.9306658e-06],
        [ 8.8922900e-07,  6.6180592e-06, -2.5627619e-06, ...,
         -3.4292477e-06,  3.3165636e-06,  4.8532293e-06]], dtype=float32), indices=array([9970, 9971, 9972, ..., 2043,   23,    1]), dense_shape=array([10000,   200])),
 array([[-1.6109510e-08,  1.7552320e-08, -1.3598928e-08, ...,
         -1.0898256e-08,  1.0346557e-09, -1.7581208e-08],
        [-

In [67]:
train_op = optimizer.apply_gradients(zip(grads, tvars))

In [68]:
session.run(tf.global_variables_initializer())
session.run(train_op, feed_dict)

In [69]:
hidden_size_l1

256

In [86]:
class PTBModel(object):
    def __init__(self, action_type):
        self.batch_size = batch_size
        self.num_steps = num_steps
        self.hidden_size_l1 = hidden_size_l1
        self.hidden_size_l2 = hidden_size_l2
        self.vocab_size = vocab_size
        self.embeding_vector_size = embeding_vector_size
        self._input_data = tf.placeholder(tf.int32, [batch_size, num_steps]) #[30#20]
        self._targets = tf.placeholder(tf.int32, [batch_size, num_steps]) #[30#20]

        ##########################################################################
        # Creating the LSTM cell structure and connect it with the RNN structure #
        ##########################################################################
        # Create the LSTM unit. 
        # This creates only the structure for the LSTM and has to be associated with a RNN unit still.
        # The argument n_hidden(size=200) of BasicLSTMCell is size of hidden layer, that is, the number of hidden units of the LSTM (inside A).
        # Size is the same as the size of our hidden layer, and no bias is added to the Forget Gate. 
        # LSTM cell processes one word at a time and computes probabilities of the possible continuations of the sentence.
        lstm_cell_l1 = tf.contrib.rnn.BasicLSTMCell(self.hidden_size_l1, forget_bias=0.0)
        lstm_cell_l2 = tf.contrib.rnn.BasicLSTMCell(self.hidden_size_l2, forget_bias=0.0)
        
        # Unless you changed keep_prob, this won't actually execute -- this is a dropout wrapper for our LSTM unit
        # This is an optimization of the LSTM output, but is not needed at all
        if action_type == "is_training" and keep_prob < 1:
            lstm_cell_l1 = tf.contrib.rnn.DropoutWrapper(lstm_cell_l1, output_keep_prob=keep_prob)
            lstm_cell_l2 = tf.contrib.rnn.DropoutWrapper(lstm_cell_l2, output_keep_prob=keep_prob)
        
        # By taking in the LSTM cells as parameters, the MultiRNNCell function junctions the LSTM units to the RNN units.
        # RNN cell composed sequentially of multiple simple cells.
        stacked_lstm = tf.contrib.rnn.MultiRNNCell([lstm_cell_l1, lstm_cell_l2])

        # Define the initial state, i.e., the model state for the very first data point
        # It initialize the state of the LSTM memory. The memory state of the network is initialized with a vector of zeros and gets updated after reading each word.
        self._initial_state = stacked_lstm.zero_state(batch_size, tf.float32)

        ####################################################################
        # Creating the word embeddings and pointing them to the input data #
        ####################################################################
        with tf.device("/cpu:0"):
            # Create the embeddings for our input data. Size is hidden size.
            embedding = tf.get_variable("embedding", [vocab_size, self.embeding_vector_size])  #[10000x200]
            # Define where to get the data for our embeddings from
            inputs = tf.nn.embedding_lookup(embedding, self._input_data)

        # Unless you changed keep_prob, this won't actually execute -- this is a dropout addition for our inputs
        # This is an optimization of the input processing and is not needed at all
        if action_type == "is_training" and keep_prob < 1:
            inputs = tf.nn.dropout(inputs, keep_prob)

        ############################################
        # Creating the input structure for our RNN #
        ############################################
        # Input structure is 20x[30x200]
        # Considering each word is represended by a 200 dimentional vector, and we have 30 batchs, we create 30 word-vectors of size [30xx2000]
        # inputs = [tf.squeeze(input_, [1]) for input_ in tf.split(1, num_steps, inputs)]
        # The input structure is fed from the embeddings, which are filled in by the input data
        # Feeding a batch of b sentences to a RNN:
        # In step 1,  first word of each of the b sentences (in a batch) is input in parallel.  
        # In step 2,  second word of each of the b sentences is input in parallel. 
        # The parallelism is only for efficiency.  
        # Each sentence in a batch is handled in parallel, but the network sees one word of a sentence at a time and does the computations accordingly. 
        # All the computations involving the words of all sentences in a batch at a given time step are done in parallel. 

        ####################################################################################################
        # Instantiating our RNN model and retrieving the structure for returning the outputs and the state #
        ####################################################################################################
        
        outputs, state = tf.nn.dynamic_rnn(stacked_lstm, inputs, initial_state=self._initial_state)
        #########################################################################
        # Creating a logistic unit to return the probability of the output word #
        #########################################################################
        output = tf.reshape(outputs, [-1, self.hidden_size_l2])
        softmax_w = tf.get_variable("softmax_w", [self.hidden_size_l2, vocab_size]) #[200x1000]
        softmax_b = tf.get_variable("softmax_b", [vocab_size]) #[1x1000]
        logits = tf.matmul(output, softmax_w) + softmax_b
        logits = tf.reshape(logits, [self.batch_size, self.num_steps, vocab_size])
        prob = tf.nn.softmax(logits)
        out_words = tf.argmax(prob, axis=2)
        self._output_words = out_words
        #########################################################################
        # Defining the loss and cost functions for the model's learning to work #
        #########################################################################
            

        # Use the contrib sequence loss and average over the batches
        loss = tf.contrib.seq2seq.sequence_loss(
            logits,
            self.targets,
            tf.ones([batch_size, num_steps], dtype=tf.float32),
            average_across_timesteps=False,
            average_across_batch=True)
    
#         loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example([logits], [tf.reshape(self._targets, [-1])],
#                                                       [tf.ones([batch_size * num_steps])])
        self._cost = tf.reduce_sum(loss)

        # Store the final state
        self._final_state = state

        #Everything after this point is relevant only for training
        if action_type != "is_training":
            return

        #################################################
        # Creating the Training Operation for our Model #
        #################################################
        # Create a variable for the learning rate
        self._lr = tf.Variable(0.0, trainable=False)
        # Get all TensorFlow variables marked as "trainable" (i.e. all of them except _lr, which we just created)
        tvars = tf.trainable_variables()
        # Define the gradient clipping threshold
        grads, _ = tf.clip_by_global_norm(tf.gradients(self._cost, tvars), max_grad_norm)
        # Create the gradient descent optimizer with our learning rate
        optimizer = tf.train.GradientDescentOptimizer(self.lr)
        # Create the training TensorFlow Operation through our optimizer
        self._train_op = optimizer.apply_gradients(zip(grads, tvars))

    # Helper functions for our LSTM RNN class

    # Assign the learning rate for this model
    def assign_lr(self, session, lr_value):
        session.run(tf.assign(self.lr, lr_value))

    # Returns the input data for this model at a point in time
    @property
    def input_data(self):
        return self._input_data


    
    # Returns the targets for this model at a point in time
    @property
    def targets(self):
        return self._targets

    # Returns the initial state for this model
    @property
    def initial_state(self):
        return self._initial_state

    # Returns the defined Cost
    @property
    def cost(self):
        return self._cost

    # Returns the final state for this model
    @property
    def final_state(self):
        return self._final_state
    
    # Returns the final output words for this model
    @property
    def final_output_words(self):
        return self._output_words
    
    # Returns the current learning rate for this model
    @property
    def lr(self):
        return self._lr

    # Returns the training operation defined for this model
    @property
    def train_op(self):
        return self._train_op

In [87]:
##########################################################################################################################
# run_one_epoch takes as parameters the current session, the model instance, the data to be fed, and the operation to be run #
##########################################################################################################################
def run_one_epoch(session, m, data, eval_op, verbose=False):

    #Define the epoch size based on the length of the data, batch size and the number of steps
    epoch_size = ((len(data) // m.batch_size) - 1) // m.num_steps
    start_time = time.time()
    costs = 0.0
    iters = 0

    state = session.run(m.initial_state)
    
    #For each step and data point
    for step, (x, y) in enumerate(reader.ptb_iterator(data, m.batch_size, m.num_steps)):
        
        #Evaluate and return cost, state by running cost, final_state and the function passed as parameter
        cost, state, out_words, _ = session.run([m.cost, m.final_state, m.final_output_words, eval_op],
                                     {m.input_data: x,
                                      m.targets: y,
                                      m.initial_state: state})

        #Add returned cost to costs (which keeps track of the total costs for this epoch)
        costs += cost
        
        #Add number of steps to iteration counter
        iters += m.num_steps

        if verbose and step % (epoch_size // 10) == 10:
            print("Itr %d of %d, perplexity: %.3f speed: %.0f wps" % (step , epoch_size, np.exp(costs / iters), iters * m.batch_size / (time.time() - start_time)))

    # Returns the Perplexity rating for us to keep track of how the model is evolving
    return np.exp(costs / iters)


In [88]:
raw_data = reader.ptb_raw_data(data_dir)
train_data, valid_data, test_data, _, _ = raw_data

In [89]:
# Initializes the Execution Graph and the Session
with tf.Graph().as_default(), tf.Session() as session:
    initializer = tf.random_uniform_initializer(-init_scale, init_scale)
    
    # Instantiates the model for training
    # tf.variable_scope add a prefix to the variables created with tf.get_variable
    with tf.variable_scope("model", reuse=None, initializer=initializer):
        m = PTBModel("is_training")
        
    # Reuses the trained parameters for the validation and testing models
    # They are different instances but use the same variables for weights and biases, they just don't change when data is input
    with tf.variable_scope("model", reuse=True, initializer=initializer):
        mvalid = PTBModel("is_validating")
        mtest = PTBModel("is_testing")

    #Initialize all variables
    tf.global_variables_initializer().run()

    for i in range(max_epoch):
        # Define the decay for this epoch
        lr_decay = decay ** max(i - max_epoch_decay_lr, 0.0)
        
        # Set the decayed learning rate as the learning rate for this epoch
        m.assign_lr(session, learning_rate * lr_decay)

        print("Epoch %d : Learning rate: %.3f" % (i + 1, session.run(m.lr)))
        
        # Run the loop for this epoch in the training model
        train_perplexity = run_one_epoch(session, m, train_data, m.train_op, verbose=True)
        print("Epoch %d : Train Perplexity: %.3f" % (i + 1, train_perplexity))
        
        # Run the loop for this epoch in the validation model
        valid_perplexity = run_one_epoch(session, mvalid, valid_data, tf.no_op())
        print("Epoch %d : Valid Perplexity: %.3f" % (i + 1, valid_perplexity))
    
    # Run the loop in the testing model to see how effective was our training
    test_perplexity = run_one_epoch(session, mtest, test_data, tf.no_op())
    
    print("Test Perplexity: %.3f" % test_perplexity)

NameError: name 'embeding_vector_size' is not defined